In [1]:
import pandas as pd

# 데이터 로드 (파일명은 필요에 따라 수정)
df = pd.read_csv('../최종사용_데이터/파생변수추가_기상과풍력.csv')

print(f"원본 데이터: {len(df):,}건")

# 풍속이 커트인 이하 또는 커트아웃 이상이면서 발전량이 0이 아닌 경우 제거
invalid_power = ((df['풍속(m/s)'] <= df['커트인']) | (df['풍속(m/s)'] >= df['커트아웃'])) & (df['발전량(kWh)'] > 0)
removed = invalid_power.sum()

# 발전구분별 제거 건수
if '발전구분' in df.columns:
    removed_by_plant = df.loc[invalid_power, '발전구분'].value_counts()
    print("발전구분별 제거 건수:")
    for plant, cnt in removed_by_plant.items():
        print(f"  {plant}: {cnt:,}건")

df = df[~invalid_power]

print(f"커트인 이하 또는 커트아웃 이상에서 발전량>0인 데이터 제거: {removed:,}건")
print(f"제거 후 데이터: {len(df):,}건")

# 결과를 같은 파일에 덮어쓰기
df.to_csv('../최종사용_데이터/파생변수추가_기상과풍력.csv', index=False, encoding='utf-8-sig')
print("덮어쓰기 완료: 파생변수추가_기상과풍력.csv")

원본 데이터: 242,378건
발전구분별 제거 건수:
  성산풍력: 2,014건
  한경풍력: 1,631건
  어음풍력: 206건
  상명풍력: 148건
커트인 이하 또는 커트아웃 이상에서 발전량>0인 데이터 제거: 3,999건
제거 후 데이터: 238,379건
덮어쓰기 완료: 파생변수추가_기상과풍력.csv


In [2]:
# 풍속이 정격이 아닌데 발전량=설비용량*1000인 경우 제거
if all(col in df.columns for col in ['풍속(m/s)', '정격', '발전량(kWh)', '설비용량(MW)']):
    cond = (df['풍속(m/s)'] != df['정격']) & (df['발전량(kWh)'] == df['설비용량(MW)'] * 1000)
    removed2 = cond.sum()
    # 발전구분-호기별 제거 건수
    if '발전구분' in df.columns and '호기' in df.columns:
        removed_by_combo = df.loc[cond].groupby(['발전구분', '호기']).size()
        print("풍속≠정격 & 발전량=설비용량*1000인 경우 발전구분-호기별 제거 건수:")
        for (plant, unit), cnt in removed_by_combo.items():
            print(f"  {plant} {unit}호기: {cnt:,}건")
    df = df[~cond]
    print(f"풍속≠정격 & 발전량=설비용량*1000인 경우 전체 제거: {removed2:,}건")
    print(f"제거 후 데이터: {len(df):,}건")
# 결과를 같은 파일에 덮어쓰기
df.to_csv('../최종사용_데이터/파생변수추가_기상과풍력.csv', index=False, encoding='utf-8-sig')
print("파생변수추가_기상과풍력.csv")

풍속≠정격 & 발전량=설비용량*1000인 경우 발전구분-호기별 제거 건수:
  성산풍력 1호기: 1건
  성산풍력 2호기: 1건
  한경풍력 1호기: 5건
  한경풍력 2호기: 18건
풍속≠정격 & 발전량=설비용량*1000인 경우 전체 제거: 25건
제거 후 데이터: 238,354건
파생변수추가_기상과풍력.csv


In [3]:
#베츠의 법칙
import numpy as np

# 변환효율 계산 및 100% 초과 제거
if all(col in df.columns for col in ['발전량(kWh)', 'air_density', '블레이드', '풍속(m/s)']):
    # kWh → Wh로 변환 (1kWh=1000Wh), 풍력공식 단위 맞추기
    gen_wh = df['발전량(kWh)'] * 1000
    rho = df['air_density']
    v = df['풍속(m/s)']
    blade = df['블레이드']
    A = np.pi * (blade ** 2)
    denom = 0.5 * rho * A * (v ** 3)
    # 0으로 나누는 경우 방지
    denom = denom.replace(0, np.nan)
    efficiency = gen_wh / denom
    cond = efficiency > 0.593
    removed3 = cond.sum()
    # 발전구분-호기별 제거 건수
    if '발전구분' in df.columns and '호기' in df.columns:
        removed_by_combo = df.loc[cond].groupby(['발전구분', '호기']).size()
        print("발전구분-호기별 제거 건수:")
        for (plant, unit), cnt in removed_by_combo.items():
            print(f"  {plant} {unit}호기: {cnt:,}건")
    df = df[~cond]
    print(f"전체 제거: {removed3:,}건")
    print(f"제거 후 데이터: {len(df):,}건")
df.to_csv('../최종사용_데이터/파생변수추가_기상과풍력.csv', index=False, encoding='utf-8-sig')

발전구분-호기별 제거 건수:
  상명풍력 1호기: 2,322건
  성산풍력 1호기: 8,357건
  성산풍력 2호기: 8,402건
  어음풍력 1호기: 4,111건
  한경풍력 1호기: 18,182건
  한경풍력 2호기: 24,842건
전체 제거: 66,216건
제거 후 데이터: 172,138건


In [4]:
# 발전구분, 호기별 데이터 개수 확인
if '발전구분' in df.columns and '호기' in df.columns:
    print("\n[INFO] 발전구분-호기별 데이터 개수:")
    counts = df.groupby(['발전구분', '호기']).size()
    for (plant, unit), cnt in counts.items():
        print(f"  {plant} {unit}호기: {cnt:,}건")


[INFO] 발전구분-호기별 데이터 개수:
  상명풍력 1호기: 12,815건
  성산풍력 1호기: 13,578건
  성산풍력 2호기: 13,389건
  어음풍력 1호기: 3,750건
  한경풍력 1호기: 66,880건
  한경풍력 2호기: 61,726건
